In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import glob as gb
import pandas as pd
import seaborn as sns
import matplotlib.image as mpimg
from tqdm import tqdm
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.optimizers import Adam


In [7]:
train_path = '/kaggle/input/raf-db-happy-sad-angry-neutral/train'
test_path = '/kaggle/input/raf-db-happy-sad-angry-neutral/test'

train_datagen = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest'
)

images_size = 100
batch_size = 64

train_generator = train_datagen.flow_from_directory(
    train_path,                  
    target_size=(images_size, images_size), 
    batch_size=batch_size,          
    seed=32,                       
    shuffle=True,                    
    class_mode='categorical'       
)

test_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.5
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(images_size, images_size),
    batch_size = batch_size,
    shuffle=False,
    class_mode='categorical')

Found 9978 images belonging to 4 classes.
Found 2505 images belonging to 4 classes.


In [8]:
# Cek struktur dataset
print(f"Train folder contains: {os.listdir(train_path)}")
print(f"Test folder contains: {os.listdir(test_path)}")

Train folder contains: ['angry', 'neutral', 'sad', 'happy']
Test folder contains: ['angry', 'neutral', 'sad', 'happy']


In [9]:
early_stopping = EarlyStopping(
    min_delta=0.001,
    patience=5,
    restore_best_weights=True,
)

learning_rate_reduce = ReduceLROnPlateau(
    monitor='val_acc',  
    patience=5,         
    verbose=1,          
    factor=0.5,          
    min_lr=0.00001       
)

from tensorflow.keras import optimizers

learning_rate_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01, 
    decay_steps=1000,            
    decay_rate=0.5,            
)

from keras.callbacks import LearningRateScheduler

lr_callback = LearningRateScheduler(learning_rate_schedule)
callback=[ lr_callback , learning_rate_reduce ,early_stopping ]

In [10]:
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from keras import regularizers
def CustomCNN():
    model= tf.keras.models.Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(images_size,images_size,3)))
    model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # model.add(Flatten()) 
    model.add(Dense(256,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
        
    model.add(Dense(512,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    return model
    # model.summary()

In [11]:
base_model = CustomCNN()
# Freeze the layers of the base modelCustom
# for layer in base_model.layers:
#     layer.trainable = False

modelCustom = tf.keras.models.Sequential([
    base_model,
    
    layers.Flatten(),
    
    layers.Dense(256,activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
        
    layers.Dense(512,activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
    layers.Dense(4, activation='softmax')
])

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

def learning_rate_schedule(epoch):
    initial_lr = 0.001
    drop_factor = 0.5
    epoch_drop = 10
    return initial_lr * (drop_factor ** (epoch // epoch_drop))

lr_callback = LearningRateScheduler(learning_rate_schedule)

optimizer = Adam()

modelCustom.compile(optimizer=optimizer,
              loss="categorical_crossentropy",
              metrics=['accuracy'])

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    '4kelasterbaek.keras',                  # Nama file untuk menyimpan model
    monitor='val_accuracy',           # Memantau akurasi validasi
    save_best_only=True,               # Hanya menyimpan model dengan performa terbaik
    mode='max',                        # Mode 'max' berarti model dengan nilai tertinggi yang akan disimpan
    verbose=1                          # Menampilkan status setiap kali model disimpan
)

history = modelCustom.fit(train_generator,
                          # steps_per_epoch=train_generator.samples // batch_size,
                          epochs=200,
                          validation_data=test_generator,
                          # validation_steps=test_generator.samples // batch_size, 
                          callbacks=[lr_callback,checkpoint]  # Use the lr_callback here
)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.4415 - loss: 8.3437
Epoch 1: val_accuracy improved from -inf to 0.53892, saving model to 4kelasterbaek.keras
156/156 ━━━━━━━━━━━━━━━━━━━━ 58s 224ms/step - accuracy: 0.4420 - loss: 8.3349 - val_accuracy: 0.5389 - val_loss: 4.5252 - learning_rate: 0.0010
Epoch 2/200
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.6354 - loss: 3.3883
Epoch 2: val_accuracy improved from 0.53892 to 0.60599, saving model to 4kelasterbaek.keras
156/156 ━━━━━━━━━━━━━━━━━━━━ 19s 116ms/step - accuracy: 0.6355 - loss: 3.3846 - val_accuracy: 0.6060 - val_loss: 2.1322 - learning_rate: 0.0010
Epoch 3/200
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.6891 - loss: 1.7250
Epoch 3: val_accuracy improved from 0.60599 to 0.64471, saving model to 4kelasterbaek.keras
156/156 ━━━━━━━━━━━━━━━━━━━━ 18s 113ms/step - accuracy: 0.6891 - loss: 1.7241 - val_accuracy: 0.6447 - val_loss: 1.4631 - learning_rate: 0.0010
Epoch 4/200
156/156 ━━━━━━━━━━